<a href="https://colab.research.google.com/github/LenaVolzhina/playing-with-neural-networks/blob/master/DL_chapter_7_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Качаем данные с википедии

In [1]:
# https://dumps.wikimedia.org/ruwiki/20191220/
!wget https://dumps.wikimedia.org/ruwiki/20191220/ruwiki-20191220-pages-articles-multistream.xml.bz2

--2019-12-24 08:01:05--  https://dumps.wikimedia.org/ruwiki/20191220/ruwiki-20191220-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3992310344 (3.7G) [application/octet-stream]
Saving to: ‘ruwiki-20191220-pages-articles-multistream.xml.bz2’

ruwiki-20191220-pag 100%[===================>]   3.72G  4.95MB/s    in 12m 48s 

2019-12-24 08:13:54 (4.96 MB/s) - ‘ruwiki-20191220-pages-articles-multistream.xml.bz2’ saved [3992310344/3992310344]



In [0]:
from datetime import datetime

# Загрузка и предобработка данных

*Токенизация*: разбиение потока символов на слова (+ очистка от лишних символов)

*Лемматизация*: приведение слов к начальным формам. Мы не будем делать, т.к. во-первых хотим посмотреть на всякие эффекты, а во-вторых, у нас и так достаточно данных

In [0]:
from gensim.corpora.wikicorpus import WikiCorpus

wiki = WikiCorpus('ruwiki-20191220-pages-articles-multistream.xml.bz2', dictionary=False)

Данные не читаются с диска в память, а благодаря dictionary=False не строится и словарь

*Биграмма* — это пара слов `(w1,w2)`, которые вместе встречаются аномально часто, то есть совместная вероятность их появления `p(w1,w2)`
в тексте существенно выше, чем `p(w1)p(w2)`

Есть разные алгоритмы выделения биграмм, но в Gensim уже есть реализация

In [0]:
def get_texts(step=100):
  # get every [step]th text
  for i, text in enumerate(wiki.get_texts()):
    if i % 100 == 0:
      yield text

In [5]:
from gensim.models.phrases import Phrases,Phraser

print(datetime.now())
bigram = Phrases(get_texts())
bigram_transformer = Phraser(bigram)
print(datetime.now())

2019-12-24 08:20:47.444497
2019-12-24 10:00:46.770970


In [0]:
# генератор биграм

def text_generator_bigram():
  for text in get_texts():
    yield bigram_transformer[ [ word for word in text ] ]   # .decode('utf-8')

In [0]:
# генератор триграм

print(datetime.now())
trigram = Phrases(text_generator_bigram())
trigram_transformer = Phraser(trigram)
print(datetime.now())

def text_generator_trigram():
  for text in get_texts():
    yield trigram_transformer[ bigram_transformer[ [ word for word in text ] ] ]   # .decode('utf-8')

2019-12-24 11:46:40.393473


# Обучаем word2vec на последовательности триграмм

In [0]:
print(datetime.now())
total_examples = 0
for trigram in text_generator_trigram():
  total_examples += 1
print(datetime.now())

In [0]:
from gensim.models.word2vec import Word2Vec

model_100 = Word2Vec(size=100, window=7, min_count=10, workers=10)
print(datetime.now())
model_100.build_vocab( text_generator_trigram() )
print(datetime.now())
model_100.train( text_generator_trigram(), total_examples=total_examples )
print(datetime.now())

* size -- размерность вектора представления
* window -- размер окна локального контекста
* min_count -- минимальная частота встречаемости, чтобы мы рассматривали слово
* workers -- параллелизация

In [0]:
model_100.wv.most_similar(
  ['математик', 'женщина'],
  ['мужчина']
)

In [0]:
model_100.wv.most_similar(
  ['актёр', 'женщина'],
  ['мужчина']
)

In [0]:
model_100.wv.most_similar(
  ['стюардесса', 'мужчина'],
  ['женщина']
)

In [0]:
from gensim.models.word2vec import Word2Vec

print(datetime.now())
model_500 = Word2Vec(size=500, window=7, min_count=10, workers=10)
print(datetime.now())
model_500.build_vocab( text_generator_trigram() )
print(datetime.now())
model_500.train( text_generator_trigram(), total_examples=total_examples )
print(datetime.now())

In [0]:
!sleep infinity